# Using AI21 Contextual Answers on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Contextual Answers** using Amazon SageMaker.

https://docs.ai21.com/docs/contextual-answers-api <BR>
https://docs.ai21.com/reference/contextual-answers-api-ref

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Interact with the model](#B.-Interact-with-the-model)
   1. [Ask about financial reports](#C.-Ask-about-financial-reports)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/contextual-answers-1-0-001-a85d7d493b3a39e3a8a8ec734f2befae"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.114'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for ai21: filename=ai21-1.1.4-py3-none-any.whl size=21794 sha256=ab61a497b0cd881938d0f345f1f10609ce126234b4f4992107785cce0ce78977
  Stored in directory: /home/ec2-user/.cache/pip/wheels/07/a4/d2/084c23e5034998226d1d532610766e00f34ca01797938a9d39
Successfully built ai21


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
endpoint_name = "contextual-answers"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:057716757052:endpoint-config/contextual-answers".

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

**AI21 Studio Contextual Answers model** allows you to access our high-quality question answering technology. It was designed to answer questions based on a specific document context provided by the customer. This avoids any factual issues that language models may have and makes sure the answers it provides are grounded in that context document.

This model receives document text, serving as a context, and a question and returns an answer based entirely on this context. This means that if the answer to your question is not in the document, the model will indicate it (instead of providing a false answer).

To get a sense of the model's behavior, let's use this toy example of asking what is the Eiffel tower height. Most language models will simply answer according to their training data.

This model, however, bases its answer solely on the context you provide. Let's use the following [Wikipedia paragraph](https://en.wikipedia.org/wiki/Eiffel_Tower#:~:text=The%20Eiffel%20Tower%20(%2F%CB%88a%C9%AA,from%20the%20Champ%20de%20Mars) as context, with small modifications:

In [ ]:
# Actual paragraph
context = "The tower is 330 metres (1,083 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with manual changes of the height
false_context = "The tower is 3 metres (10 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with the height omitted
partial_context = "Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

Here is what the model will say when asked the same question in each context.

In [ ]:
# True context
response = ai21.Answer.execute(
    context=context,
    question="What is the height of the Eiffel tower?",
    sm_endpoint=endpoint_name
)

print(response.answer)

In [ ]:
# False context
response = ai21.Answer.execute(
    context=false_context,
    question="What is the height of the Eiffel tower?",
    sm_endpoint=endpoint_name
)

print(response.answer)

In [ ]:
# Irrelevant context
response = ai21.Answer.execute(
    context=partial_context,
    question="What is the height of the Eiffel tower?",
    sm_endpoint=endpoint_name
)

print(response.answer)

### C. Ask about financial reports

The document context should be **no more than 10,000 characters**, and the question can be up to 160 characters.

Imagine you are performing research and rely on financial reports to base your findings. Let's take the following part from [JPMorgan Chase & Co. 2021 annual report](https://www.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/investor-relations/documents/annualreport-2021.pdf):

In [ ]:
financial_context = """In 2020 and 2021, enormous QE — approximately $4.4 trillion, or 18%, of 2021 gross domestic product (GDP) — and enormous fiscal stimulus (which has been and always will be inflationary) — approximately $5 trillion, or 21%, of 2021 GDP — stabilized markets and allowed companies to raise enormous amounts of capital. In addition, this infusion of capital saved many small businesses and put more than $2.5 trillion in the hands of consumers and almost $1 trillion into state and local coffers. These actions led to a rapid decline in unemployment, dropping from 15% to under 4% in 20 months — the magnitude and speed of which were both unprecedented. Additionally, the economy grew 7% in 2021 despite the arrival of the Delta and Omicron variants and the global supply chain shortages, which were largely fueled by the dramatic upswing in consumer spending and the shift in that spend from services to goods. Fortunately, during these two years, vaccines for COVID-19 were also rapidly developed and distributed.
In today's economy, the consumer is in excellent financial shape (on average), with leverage among the lowest on record, excellent mortgage underwriting (even though we've had home price appreciation), plentiful jobs with wage increases and more than $2 trillion in excess savings, mostly due to government stimulus. Most consumers and companies (and states) are still flush with the money generated in 2020 and 2021, with consumer spending over the last several months 12% above pre-COVID-19 levels. (But we must recognize that the account balances in lower-income households, smaller to begin with, are going down faster and that income for those households is not keeping pace with rising inflation.)
Today's economic landscape is completely different from the 2008 financial crisis when the consumer was extraordinarily overleveraged, as was the financial system as a whole — from banks and investment banks to shadow banks, hedge funds, private equity, Fannie Mae and many other entities. In addition, home price appreciation, fed by bad underwriting and leverage in the mortgage system, led to excessive speculation, which was missed by virtually everyone — eventually leading to nearly $1 trillion in actual losses.
"""

Rather than reading the entire report, just ask what you want to know:

In [ ]:
question = "Did the economy shrink after the Omicron variant arrived?"

The model will answer based on the provided report:

In [9]:
response = ai21.Answer.execute(
    context=financial_context,
    question=question,
    sm_endpoint=endpoint_name
)

print(response.answer)

NameError: name 'financial_context' is not defined

In addition, you can ask more complex questions, where the answer requires deductions rather than just extracting the correct sentence from the document context. This will result in abstractive, rather than extractive, answers that draw on several different parts of the document. For example, look at the following question:

In [28]:
harder_question = "Did COVID-19 eventually help the economy?"

response = ai21.Answer.execute(
    context=financial_context,
    question=harder_question,
    sm_endpoint=endpoint_name
)

print(response.answer)

The COVID-19 pandemic had a significant short-term impact on the economy, as it led to shutdowns and layoffs across the country. However, it also had a long-term positive impact by accelerating digital transformation and encouraging innovation.


We now present the model with the following question. You may be confused to answer something based on the last paragraph without delving into the text. However, if you read the provided document context properly, you will discover that the answer does not appear there. The model handles this as expected:

In [21]:
irrelevant_question = "How did COVID-19 affect the financial crisis of 2008?"

response = ai21.Answer.execute(
    context=financial_context,
    question=irrelevant_question,
    sm_endpoint=endpoint_name
)

print(response.answer)

Answer not in document


### D. Translate

In [30]:
import boto3

In [31]:
translate = boto3.client("translate")

In [94]:
def trans(text, target="ko"):
    response = translate.translate_text(
        Text=text,
        SourceLanguageCode="Auto",
        TargetLanguageCode=target
    )
    
    text_translate = response["TranslatedText"]
    #print (text_translate)
    return text_translate

In [55]:
financial_context_ko = """2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오늘날의 경제 상황은 2008년 금융 위기와 완전히 달라졌습니다. 은행과 투자 은행에서부터 섀도우 뱅크, 헤지 펀드, 사모펀드, 패니 매 (Fannie Mae) 및 기타 여러 기업에 이르기까지 금융 시스템 전체가 그랬듯이 소비자가 과도하게 레버리지를 받았을 때였습니다.또한 주택 담보 대출 제도의 부실 인수 및 레버리지로 인한 주택 가격 상승은 과도한 투기로 이어져 거의 모든 사람이 이를 놓쳤고 결국 거의 1조 달러에 가까운 실제 손실을 초래했습니다."""

2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.<BR>
또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.<BR>
이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.<BR>
**또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다.** <BR>
이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.<BR>
다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.<BR>
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다.<BR>
이는 주로 정부 경기 부양책으로 인한 것입니다.<BR>
대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.
(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)<BR>
오늘날의 경제 상황은 2008년 금융 위기와 완전히 달라졌습니다.<BR>
은행과 투자 은행에서부터 섀도우 뱅크, 헤지 펀드, 사모펀드, 패니 매 (Fannie Mae) 및 기타 여러 기업에 이르기까지 금융 시스템 전체가 그랬듯이 소비자가 과도하게 레버리지를 받았을 때였습니다.<BR>
또한 주택 담보 대출 제도의 부실 인수 및 레버리지로 인한 주택 가격 상승은 과도한 투기로 이어져 거의 모든 사람이 이를 놓쳤고 결국 거의 1조 달러에 가까운 실제 손실을 초래했습니다.<BR>

In [56]:
question = "오미크론 변종이 등장한 후 경제가 위축되었나요?"

In [57]:
response = ai21.Answer.execute(
    context=trans(financial_context_ko, target="en"),
    question=trans(question, target="en"),
    sm_endpoint=endpoint_name
)
response_ko = trans(response.answer)
print(response_ko)

아니요, 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다.


In [95]:
harder_question = "COVID-19가 결국 경제에 도움이 되었나요?"

response = ai21.Answer.execute(
    context=trans(financial_context_ko, target="en"),
    question=trans(harder_question, target="en"),
    sm_endpoint=endpoint_name,
    temperature=1.1
)
response_ko = trans(response.answer)
print(response_ko)

COVID-19 백신도 이 2년 동안 빠르게 개발 및 보급되었으며, 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다.


In [91]:
irrelevant_question = "How did COVID-19 affect the financial crisis of 2008?"

response = ai21.Answer.execute(
    context=trans(financial_context_ko, target="en"),
    question=trans(irrelevant_question, target="en"),
    sm_endpoint=endpoint_name,
    temperature=0.7
)

response_ko = trans(response.answer)
print(response_ko)

In 2020 and 2021, huge QE of about 4.4 trillion dollars, or 18% of gross domestic product (GDP) in 2021, and huge fiscal stimulus measures of about 5 trillion dollars, or 21% of GDP in 2021 (inflation will still be inflation until now) stabilized the market, and companies were able to raise huge amounts of capital. Furthermore, these capital inputs saved many small and medium-sized enterprises, and invested more than 2.5 trillion dollars in consumers and almost 1 trillion dollars in state and local vaults. The unemployment rate was 20 due to these measures. It declined sharply from 15% to less than 4% in just a month. The scale and pace were unprecedented. Furthermore, the economy grew 7% in 2021 despite the advent of the Delta and Omicron variants and global supply chain shortages. This was mainly driven by a sharp increase in consumer spending and a shift in that spending from services to goods. Fortunately, COVID-19 vaccines were also rapidly developed and distributed during these t

In [68]:
ai21.Answer.execute?

Signature: ai21.Answer.execute(**params)
Docstring: <no docstring>
File:      ~/anaconda3/envs/python3/lib/python3.10/site-packages/ai21/modules/resources/nlp_task.py
Type:      method

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/contextual-answers-api) to understand all the capabilities of AI21 Contextual Answers model

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()